In [143]:
import dspy
import os
import base64
import requests
from PIL import Image
from io import BytesIO
from openai import OpenAI
from pdf2image import convert_from_path

In [144]:
#convert pdf to base64 image for GPT input
import base64
from pdf2image import convert_from_path
from io import BytesIO

def pdf_to_base64_image(pdf_path, first_page=1, last_page=1, image_format='JPEG', quality=100):
    # Convert the specified PDF pages to images
    images = convert_from_path(pdf_path, first_page=first_page, last_page=last_page)
    
    # Convert the first image to base64
    buffer = BytesIO()  # Create an in-memory buffer
    images[0].save(buffer, format=image_format, quality=quality)  # Save the image to the buffer
    buffer.seek(0)  # Reset buffer position to the start
    
    return base64.b64encode(buffer.read()).decode('utf-8')  # Encode to base64 and return

In [145]:
# Call gpt to parse image
def get_csv_tabel(contract_base64):
  api_key = api_key=os.environ.get("OPENAI_API_KEY")
  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4o",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Parse the image and extract the table. Specify the column names and return each row as a new line, with columns separated by a semicolon (';'). Only return the table without any additional text or explanation."
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{contract_base64}"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  text = response.json()['choices'][0]['message']['content']

  return text


In [140]:
# Convert image to structured output
from io import StringIO
import pandas as pd

pdf_path = 'pdf/187716-1 Klantorder definitief.pdf'

base64_image = pdf_to_base64_image(pdf_path, first_page=3, last_page=3)
text = get_csv_tabel(base64_image)
text = text.lstrip('```\n')
csv_data = StringIO(text)
df = pd.read_csv(csv_data, delimiter=';')

df

,Aantal,Artikelcode,Kleur,Omschrijving,Bedrag
0,1 st,VTVW,NaN,"Begane grond, Vloertegels De vloertegels in wi...",
1,1 st,VTB,NaN,De vloertegels Busy 706 invoegen,
2,1 st,VIPL,NaN,Geen plintafwerking opgenomen tbv aansluiting ...,
3,1 st,VTKEU,NaN,De vloertegels ook onder de keuken doorleggen,
4,1 st,VTBOMR,NaN,De matrandomranding in hal rondom afwerken met...,
5,61.80 m²,1442553,White,Vloertegels 250 x 1500 x 11 mm PorcST 250X1500...,
6,46.8 m²,TVB099,NaN,"Het aanbrengen van vloertegels hal, keuken of ...",
7,1 st,NaN,NaN,NaN,NaN


## DSPy

In [154]:
# Set up the LM
gpt_4o = dspy.OpenAI(model='gpt-4o')
dspy.settings.configure(lm=gpt_4o)

In [155]:
class GenerateQuestion(dspy.Signature):
    """Generate a question that must be validated, based on the 'omschrijving' column of the first row."""

    table = dspy.InputField(desc="Table with 'omschrijving' column")
    question = dspy.OutputField(desc="a question that must be validated")

class ValidateQuestion(dspy.Signature):
    """Validate if the question can be answered"""

    question = dspy.InputField(desc="a question")
    answer = dspy.OutputField(desc="yes or no")

In [157]:
class Questioner(dspy.Module):
    def __init__(self):
         super().__init__()
         self.generate_questions = dspy.ChainOfThought(GenerateQuestion)

    def forward(self, contract):
         table = get_csv_tabel(contract)
         question = self.generate_questions(table=table)

         return question
    
class Validator(dspy.Module):
    def __init__(self):
         super().__init__()
         self.validate_question = dspy.ChainOfThought(ValidateQuestion)

    def forward(self, contract):
         question = pred.question
         answer = self.validate_question(question=question)

         return answer

In [164]:
question_agent = Questioner()
pred = question_agent(base64_image)

validate_agent = Validator()
val = validate_agent(pred.question)

print(val.answer)

no


In [165]:
gpt_4o.inspect_history()




Validate if the question can be answered

---

Follow the following format.

Question: a question
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: yes or no

---

Question: Is there an attached drawing that clearly indicates the laying direction for processing the floor tiles in a random pattern as described?
Reasoning: Let's think step by step in order to produce the answer. We need to determine if there is an attached drawing that provides a clear indication of the laying direction for the floor tiles. Since the question specifically asks about the presence of an attached drawing, we need to check for any attachments or references to a drawing that accompanies the question. Without access to the actual content or context, we cannot verify the presence of such a drawing. Therefore, we cannot definitively answer the question based solely on the information provided.

Answer: no





"\n\n\nValidate if the question can be answered\n\n---\n\nFollow the following format.\n\nQuestion: a question\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: yes or no\n\n---\n\nQuestion: Is there an attached drawing that clearly indicates the laying direction for processing the floor tiles in a random pattern as described?\nReasoning: Let's think step by step in order to\x1b produce the answer. We need to determine if there is an attached drawing that provides a clear indication of the laying direction for the floor tiles. Since the question specifically asks about the presence of an attached drawing, we need to check for any attachments or references to a drawing that accompanies the question. Without access to the actual content or context, we cannot verify the presence of such a drawing. Therefore, we cannot definitively answer the question based solely on the information provided.\n\nAnswer: no\x1b\n\n\n"